In [1]:
import pandas as pd
import csv
import os

In [17]:
path="C:\\Users\\Olga\\Desktop\\docs\\hackathon\\"
filename="with_reference.xlsx"

In [18]:
wr=pd.ExcelFile(path+filename)

In [25]:
a[0].columns=a[0].columns.str.lower()

In [26]:
a=[]
for i in range(len(wr.sheet_names)):
    a.append(wr.parse(wr.sheet_names[i]))
    a[i].columns=a[i].columns.str.lower()
a[1].head()

,id,rate
0,313d1da6-b871-4c3c-893b-54bd1c5b8cec,3.0
1,5a3eb9fa-f7b5-4e44-8506-c4359050f980,2.7
2,1f1f54f1-6263-496c-9dbf-72fa5deef60f,2.4
3,2e88dfaa-e4ae-438b-90b3-b9af1e99e789,2.1
4,f1d64162-1af0-4f79-b9a3-f5247ea07809,1.8


In [27]:
a[0].rename(columns = {'rate id':'id'}, inplace = True)
df_merged = pd.merge(a[0], 
              a[1], 
              on ='id', 
              how ='left')

In [28]:
df_merged.head()

,base,id,period start,period end,fee,rate
0,0,313d1da6-b871-4c3c-893b-54bd1c5b8cec,2021-02-17,2021-10-14,0.00,3.0
1,4587,f1d64162-1af0-4f79-b9a3-f5247ea07809,2021-07-13,2021-08-19,8.37,1.8
2,470,313d1da6-b871-4c3c-893b-54bd1c5b8cec,2021-09-05,2021-11-23,3.05,3.0
3,9347,240ee602-ad4f-4792-8ca8-ab813a2ea6b3,2021-05-20,2021-09-13,8.91,0.3
4,8310,1e760cd7-30f5-4e7b-a77e-3e678116a82a,2021-01-13,2021-01-31,2.46,0.6


In [36]:
from datetime import datetime
date_format = '%Y-%m-%d %H:%M:%S'
df_merged['period_days']=df_merged.apply(lambda x: (datetime.strptime(str(x['period end']), date_format)-datetime.strptime(str(x['period start']), date_format)).days, axis=1)
df_merged['formula']=df_merged.apply(lambda x: (x['base']*x['period_days']/365)*(x['rate']/100), axis=1)
df_merged['diff_formula']=df_merged.apply(lambda x: abs(x['formula']-x['fee']),axis=1)
df_merged.sort_values(by=['diff_formula'], ascending=False).head()

,base,id,period start,period end,fee,rate,period_days,mult,diff,formula,diff_formula
77,8900,1e760cd7-30f5-4e7b-a77e-3e678116a82a,2021-07-17,2021-08-09,3.36,0.6,23,53.40,50.04,3.364932,0.004932
19,8977,1e760cd7-30f5-4e7b-a77e-3e678116a82a,2021-11-27,2021-11-29,0.30,0.6,2,53.86,53.56,0.295134,0.004866
24,2771,1f1f54f1-6263-496c-9dbf-72fa5deef60f,2021-11-29,2021-12-15,2.92,2.4,16,66.50,63.58,2.915244,0.004756
88,7640,098774c6-a834-4213-a2d3-b35e8480851f,2021-08-18,2021-09-15,5.27,0.9,28,68.76,63.49,5.274740,0.004740
73,191,313d1da6-b871-4c3c-893b-54bd1c5b8cec,2021-11-19,2021-12-11,0.35,3.0,22,5.73,5.38,0.345370,0.004630


In [40]:
max(df_merged['diff_formula'])<0.02

True

In [105]:
wpd['formula']=wpd.apply(lambda x: (x['base']*x['period_days']/365)*(x['rate']/100), axis=1)
wpd['diff_formula']=wpd.apply(lambda x: abs(x['formula']-x['fee']),axis=1)
wpd.sort_values(by=['diff'], ascending=False).head()
#multiplication with taking number of days into account does work!

,base,rate,period start,period end,fee,mult,diff,period_days,formula,diff_formula
77,8900,0.6,2021-07-17,2021-08-09,3.36,53.40,0.004932,23,3.364932,0.004932
19,8977,0.6,2021-11-27,2021-11-29,0.30,53.86,0.004866,2,0.295134,0.004866
24,2771,2.4,2021-11-29,2021-12-15,2.92,66.50,0.004756,16,2.915244,0.004756
88,7640,0.9,2021-08-18,2021-09-15,5.27,68.76,0.004740,28,5.274740,0.004740
73,191,3.0,2021-11-19,2021-12-11,0.35,5.73,0.004630,22,0.345370,0.004630


In [44]:
print('formula: base*((period end-period start)/365)*(rate/100)')

formula: base*((period end-period start)/365)*(rate/100)


In [45]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [91]:
y=wpd['fee']
X=wpd[['base','rate','period_days']]
pr = LinearRegression()
mse=[]
r2=[]
for i in range(9):
    pol = PolynomialFeatures(degree=i)
    X_quad = pol.fit_transform(X)
    pr = LinearRegression()
    pr.fit(X_quad, y)
    y_pred = pr.predict(X_quad)
    mse.append(mean_squared_error(y, y_pred))
    r2.append(pr.score(X_quad, y))
 
pd.DataFrame({'Polynom':range(9),'MSE':mse, 'R2':r2}).sort_values(by='MSE')

In [93]:
pd.DataFrame({'Polynom':range(9),'MSE':mse, 'R2':r2}).sort_values(by='MSE')

,MSE,Polynom,R2
3,0.000006,3,1.000000
4,0.000007,4,1.000000
5,0.045738,5,0.999753
6,0.462297,6,0.997504
7,1.060964,7,0.994272
8,3.436859,8,0.981445
2,15.439454,2,0.916644
1,39.844595,1,0.784884
0,185.224049,0,0.000000


In [80]:

y_pred = pr.predict(X_quad)
print('MSE:', mean_squared_error(y, y_pred))

print('polynomial reg R2:', pr.score(X_quad, y))
#print('liniar R2:', lr.score(X, y))

MSE: 5.945101029188217e-06
polynomial reg R2: 0.9999999679031903


In [81]:
pr.coef_

array([ 0.00000000e+00,  9.49028381e-05,  2.03345499e-02,  1.38296071e-02,
       -1.14602636e-08, -2.45229083e-05, -2.63172146e-06,  6.27900677e-02,
       -8.85615062e-03, -1.88895011e-06,  4.28934638e-13,  2.05258307e-09,
        1.25215772e-10, -2.42783983e-06,  2.82409936e-05,  1.58440719e-10,
       -1.44281243e-02,  1.40640918e-03,  7.27646315e-07, -2.45167178e-10])

In [96]:
filename='simple.xlsx'
simple=pd.ExcelFile(path+filename)

In [105]:
simple.parse(simple.sheet_names[0]).head()

,base,rate,fee
0,0,3.0,0.00
1,1315,2.7,35.51
2,7556,0.9,68.00
3,4587,1.8,82.57
4,5328,1.5,79.92


In [108]:
s=simple.parse(simple.sheet_names[0])

In [110]:
y=s['fee']
X=s[['base','rate']]
pr = LinearRegression()
mse=[]
r2=[]
for i in range(9):
    pol = PolynomialFeatures(degree=i)
    X_quad = pol.fit_transform(X)
    pr = LinearRegression()
    pr.fit(X_quad, y)
    y_pred = pr.predict(X_quad)
    mse.append(mean_squared_error(y, y_pred))
    r2.append(pr.score(X_quad, y))
 
pd.DataFrame({'Polynom':range(9),'MSE':mse, 'R2':r2}).sort_values(by='MSE')

,MSE,Polynom,R2
3,0.000006,3,1.000000
2,0.000007,2,1.000000
4,0.000008,4,1.000000
5,0.247566,5,0.999574
6,28.832606,6,0.950360
7,72.043473,7,0.875965
8,126.047146,8,0.782989
1,546.075664,1,0.059841
0,580.833415,0,0.000000
